<a href="https://colab.research.google.com/github/seanlee10/gen-ai-playground/blob/main/youtube_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU yt-dlp boto3 deepgram-sdk langchain-aws langchain-core langchain-groq langchain-nvidia-ai-endpoints langchain-anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.

In [ ]:
import re
from yt_dlp import YoutubeDL

video_url = input('Enter the video url: ')

def to_snake_case(string):
    # Replace any non-word character with an underscore
    s1 = re.sub(r'[^\w]+', '_', string)
    # Insert an underscore before any uppercase letter which is preceded by a lowercase letter or number
    s2 = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', s1)
    # Convert to lowercase
    return s2.lower()

def download_audio(link):
  with YoutubeDL({'extract_audio': True, 'format': 'bestaudio', 'outtmpl': 'download.mp3'}) as video:
    info_dict = video.extract_info(link, download = True)
    video_title = to_snake_case(info_dict['title'])
    print(video_title)
    video.download(link)
    print("Successfully Downloaded - see local folder on Google Colab")

    return video_title

video_title = download_audio(video_url)

Enter the video url: https://www.youtube.com/watch?v=IxkvVZua28k
[youtube] Extracting URL: https://www.youtube.com/watch?v=IxkvVZua28k
[youtube] IxkvVZua28k: Downloading webpage
[youtube] IxkvVZua28k: Downloading ios player API JSON
[youtube] IxkvVZua28k: Downloading mweb player API JSON
[youtube] IxkvVZua28k: Downloading player baafab19
[youtube] IxkvVZua28k: Downloading m3u8 information
[info] IxkvVZua28k: Downloading 1 format(s): 251
[download] Destination: download.mp3
[download] 100% of   40.75MiB in 00:00:03 at 12.39MiB/s  
a_conversation_with_open_ai_s_cpo_kevin_weil_anthropic_s_cpo_mike_krieger_and_sarah_guo
[youtube] Extracting URL: https://www.youtube.com/watch?v=IxkvVZua28k
[youtube] IxkvVZua28k: Downloading webpage
[youtube] IxkvVZua28k: Downloading ios player API JSON
[youtube] IxkvVZua28k: Downloading mweb player API JSON
[youtube] IxkvVZua28k: Downloading m3u8 information
[info] IxkvVZua28k: Downloading 1 format(s): 251
[download] download.mp3 has already been downloaded

In [ ]:
from deepgram import (
    DeepgramClient,
    DeepgramClientOptions,
    PrerecordedOptions,
    FileSource,
)
from deepgram.utils import verboselogs
import httpx
from datetime import datetime
from google.colab import userdata

AUDIO_FILE = 'download.mp3'

try:
    # STEP 1 Create a Deepgram client using the API key in the environment variables
    deepgram: DeepgramClient = DeepgramClient(api_key=userdata.get('deepgram'))
    # OR use defaults
    # deepgram: DeepgramClient = DeepgramClient()

    # STEP 2 Call the transcribe_file method on the rest class
    with open(AUDIO_FILE, "rb") as file:
        buffer_data = file.read()

    payload: FileSource = {
        "buffer": buffer_data,
    }

    options: PrerecordedOptions = PrerecordedOptions(
        model="nova-2",
        smart_format=True,
    )

    before = datetime.now()
    response = deepgram.listen.rest.v("1").transcribe_file(
        payload, options, timeout=httpx.Timeout(300.0, connect=10.0)
    )
    after = datetime.now()

    print(response.to_json(indent=4))
    print("")
    difference = after - before
    print(f"time: {difference.seconds}")

except Exception as e:
    print(f"Exception: {e}")


In [ ]:
transcription = response['results']['channels'][0]['alternatives'][0]['transcript']



In [ ]:
import boto3
from google.colab import userdata

session = boto3.Session(
    aws_access_key_id=userdata.get('aws_access_key_id'),
    aws_secret_access_key=userdata.get('aws_secret_access_key'),
    region_name="ap-northeast-2"
)

s3 = session.client('s3')
transcribe = session.client('transcribe')

local_file = 'download.mp3'
bucket = 'cdk-hnb659fds-assets-172054518491-ap-northeast-2'
s3_file = f"{video_title}.mp3"

def upload_to_s3(local_file, bucket, s3_file):
    s3 = session.client('s3')

    try:
        s3.upload_file(local_file, bucket, s3_file)
        print(f"Upload Successful: {local_file} uploaded to {bucket}/{s3_file}")
        return True
    except FileNotFoundError:
        print(f"The file {local_file} was not found")
        return False

uploaded = upload_to_s3(local_file, bucket, s3_file)

if uploaded:
    print("File upload completed")
else:
    print("File upload failed")

Upload Successful: download.mp3 uploaded to cdk-hnb659fds-assets-172054518491-ap-northeast-2/elon_s_predictions_for_the_future_ai_free_energy_and_more_ep_129.mp3
File upload completed


In [ ]:
def create_transcribe_job(job_name, audio_file_uri):
    response = transcribe.start_transcription_job(
        TranscriptionJobName=job_name,
        LanguageCode='en-US',  # Specify the language code
        MediaFormat='mp3',  # Specify the media format (e.g., mp3, wav)
        Media={
            'MediaFileUri': audio_file_uri
        },
        Settings={
            'ShowSpeakerLabels': True,
            'MaxSpeakerLabels': 3
        }
    )

    return response

# Example usage
job_name = video_title
audio_file_uri = f"s3://{bucket}/{s3_file}"

response = create_transcribe_job(job_name, audio_file_uri)
print(f"Transcription job created: {response['TranscriptionJob']['TranscriptionJobName']}")

Transcription job created: elon_s_predictions_for_the_future_ai_free_energy_and_more_ep_129


In [ ]:
response = transcribe.get_transcription_job(
    TranscriptionJobName=job_name
)

response['TranscriptionJob']['TranscriptionJobStatus']

'COMPLETED'

In [ ]:
import requests
import boto3
from google.colab import userdata

def get_transcription_result(job_name):
    job = transcribe.get_transcription_job(
        TranscriptionJobName=job_name
    )

    transcript_uri = job['TranscriptionJob']['Transcript']['TranscriptFileUri']

    print(transcript_uri)
    # Parse S3 URI to get bucket and key
    from urllib.parse import urlparse
    parsed_uri = urlparse(transcript_uri)
    bucket = parsed_uri.netloc
    key = parsed_uri.path.lstrip('/')

    # Download the transcription file
    response = requests.get(transcript_uri)
    if response.status_code == 200:
        transcript = response.json()

        # Extract the transcription text
        transcription_text = transcript['results']['transcripts'][0]['transcript']
        return transcription_text
    else:
        return f"Failed to download transcript. Status code: {response.status_code}"

transcription = get_transcription_result(job_name)
print(len(transcription))

https://s3.ap-northeast-2.amazonaws.com/aws-transcribe-ap-northeast-2-prod/172054518491/elon_s_predictions_for_the_future_ai_free_energy_and_more_ep_129/86785581-8796-4bd5-9cae-c441d09b47d3/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEOH%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaDmFwLW5vcnRoZWFzdC0yIkcwRQIgL7mITV5ERR6W1nAr2Hc5TMXnWc0v6FSsaWbFB2IPdm0CIQDqmX%2FRM3YXtPSHhs%2Fy%2F8YZTfOFHRIX2qSRRR1CNpNn%2FSrKBQhqEAMaDDQzNTc2MzIxMDc3NiIMruux8t46K%2BneMfaTKqcFWm2phwGzFHsoAmROZws0ZVN3VcP4O2KyE21Yukb3csuEXmmdNsIsvCVONDVYZv0ybanV0F4chxesXXVpQYgiDPLbQuUoP1wXeFfZMp2wYWTiva2vVSrwaGUlL%2BG5S%2FHLbyA2%2Fq3FjoN5G%2BKbVVzBvbaOWmYbDjYbNP6Xe2dADLl3OWdpwJIMug8RC6DR7nTsGZIUduz2VyMjrji9qQiSRV0YC716A5fPbqWKuRD45ncwDi0cL9qBeDgIY5hhHpni7ycP2VwWHpXj4IP77Veb%2BaPy9Q2K3uLR49jMA%2BLMgyweXNzYKNoJAiMCIxffOWL1NVSjgyPh84X5LS9gpUsA8CJS%2FsJT88gIbGe1ahjesJdcjcxrsfQ7ofIVBroLp0%2BDNOuclVhLsxVo89f2M317X0pi026W0tTuADzKSPyBp8GafEDLv1GVKGCpEGnURO3itGaeRwTTw%2F8HTPZFDzPWTdzCZ04jhxHnHKwJE4O0JHTsbSqTHE0BL7PYOFoLzSOCb8uapd11KJB9E%

In [ ]:
with open(f"{video_title}.txt", "w") as text_file:
    text_file.write(transcription)

In [ ]:
with open("transcript.md", "r") as transcript:
    transcription = transcript.read()

print(transcription)

Welcome to Google DeepMind, the podcast with me, your host, Professor Hannah Fry. Now, when we first started thinking about making this podcast, way back in 2017, DeepMind was this relatively small focused AI research lab. They’d just been bought by Google and given the freedom to do their own quirky research projects from the safe distance of London. Well, how things have changed? Because since the last season, Google has reconfigured its entire structure, putting AI and the team at DeepMind at the core of its strategy. Google DeepMind has continued its quest to endow AI with human level intelligence, known as artificial general intelligence or AGI. It has introduced a family of powerful new AI models called Gemini, as well as an AI agent called Project Astra that can process audio, video, image and code. The lab is also making huge leaps in applying AI to a host of scientific domains, including a brand new third version of AlphaFold which can predict the structures of all of the mole

In [ ]:
import os
import boto3
from langchain_aws import ChatBedrock
from langchain_groq import ChatGroq
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata

# session = boto3.Session(
#     aws_access_key_id=userdata.get('aws_access_key_id'),
#     aws_secret_access_key=userdata.get('aws_secret_access_key'),
#     region_name="ap-northeast-2"
# )

# BEDROCK_CLIENT = session.client("bedrock-runtime", 'ap-northeast-2')

# llm = ChatBedrock(
#     model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
#     model_kwargs=dict(temperature=0),
#     client=BEDROCK_CLIENT,
# )

# os.environ["GROQ_API_KEY"] = userdata.get('groq')

# llm = ChatGroq(
#     model="llama-3.1-70b-versatile",
#     temperature=0,
#     max_tokens=None,
# )

# os.environ["NVIDIA_API_KEY"] = userdata.get("nvidia")

# llm = ChatNVIDIA(
#     model="nvidia/llama-3.1-nemotron-70b-instruct"
# )

# os.environ["ANTHROPIC_API_KEY"] = userdata.get('anthropic')

# llm = ChatAnthropic(
#     model="claude-3-5-sonnet-20240620",
#     temperature=0,
# )

# transcription = response['results']['channels'][0]['alternatives'][0]['transcript']


# template = """
# You are tasked with summarizing a YouTube video lecture. Your goal is to create a concise and informative summary that captures the main points and key takeaways from the lecture. Follow these instructions carefully to produce an effective summary.

# Here is the transcript of the video lecture:

# <transcript>
# {TRANSCRIPT}
# </transcript>

# To summarize the lecture, please follow these steps:

# 1. Carefully read through the entire transcript.
# 2. Identify the main topic or theme of the lecture.
# 3. Determine the key points, arguments, or concepts presented in the lecture.
# 4. Note any important examples, case studies, or data that support the main ideas.
# 5. Identify the overall structure or flow of the lecture.

# Create your summary using the following format:

# <summary>
# Title: [Provide a concise title that captures the main topic of the lecture]

# Main Topic: [Briefly state the overarching subject or theme of the lecture in 1-2 sentences]

# Key Points:
# 1. [First key point]
# 2. [Second key point]
# 3. [Third key point]
#    [Add more points as needed, aiming for 3-5 in total]

# Supporting Information:
# - [Include 1-3 important examples, case studies, or pieces of data that reinforce the key points]

# Conclusion: [Summarize the main takeaway or conclusion of the lecture in 1-2 sentences]
# </summary>

# Additional instructions:
# - Keep your summary concise, aiming for about 200-300 words in total.
# - Use clear and straightforward language.
# - Focus on the most important information; avoid including minor details or tangential points.
# - Ensure that your summary accurately reflects the content of the lecture without adding any external information or personal opinions.
# - If the lecture contains technical terms or jargon, briefly explain them if they are crucial to understanding the main points.

# Remember, your goal is to provide a clear and informative overview that would help someone quickly understand the main content of the lecture without watching the entire video.
# """

template = """
I'd like to write a blog post with the insights from the provided transcript. list all important points with supporting quotes. Your answer must be in Korean.

<transcript>
{TRANSCRIPT}
</transcript>
"""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | llm | (lambda x: x.content)

summary = chain.invoke({"TRANSCRIPT": transcription})
print(summary)


TimeoutException: Requesting secret grok timed out. Secrets can only be fetched when running from the Colab UI.

In [ ]:
import requests
import json
from google.colab import userdata

auth_token = userdata.get('notion')

headers = {
    "Authorization": f"Bearer {auth_token}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-02-22"
}

payload = {
	  "parent": { "page_id": "11d11dd8536b801385c2d7d3faa6e8a1" },
    "properties": {
        "Name": {
            "title": [
                {
                    "text": {
                        "content": "Conversation with Groq CEO Jonathan Ross"
                    }
                }
            ]
        }
    },
    "children": [
		{
			"object": "block",
			"type": "heading_2",
			"heading_2": {
				"rich_text": [{ "type": "text", "text": { "content": "Lacinato kale" } }]
			}
		},
		{
			"object": "block",
			"type": "paragraph",
			"paragraph": {
				"rich_text": [
					{
						"type": "text",
						"text": {
							"content": transcription,
						}
					}
				]
			}
		}
	]
}

# print(payload)

try:
    response = requests.post("https://api.notion.com/v1/pages", headers=headers, data=payload)
    response.raise_for_status()
    print(response.json())
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


An error occurred: 400 Client Error: Bad Request for url: https://api.notion.com/v1/pages
